interesting directories: 
Other
Physical Activity 
Sleep 
Stress

Things to explore:
1. how to save the data between uses (so I don't have to read from file every time) 

    pickling? 
    creating a csv? 
    creating a sql database to pull in? 

plan: 
keep tables (bpm, sleep, activity, sleep temperature etc.) separate until I get a daily summary for each activity 
merge all the tables together using date as the primary key 
daily sleep will be the response variabel 
everything else will be explanatory variables 


In [17]:
import os

# this directoy only contains data relating to physical activity. Other directories 
# like account information and fitbit social activity 
path = "/Users/jackiekinsler/projects/sleep_analysis_py/physical_data"
 
# to store metric types in a set
unique_metrics = set()

# dirs=directories
for (root, dirs, file) in os.walk(path):
    for f in file:
        if f.endswith(".csv"):
            metric = f.split("-")[0]
            unique_metrics.add(metric)
        
for metric in unique_metrics:
    print(metric)

Respiratory Rate Summary 
Stress Score.csv
Daily SpO2 
Computed Temperature 
sleep_score.csv
Daily Respiratory Rate Summary 
estimated_oxygen_variation
Active Zone Minutes 
Daily Heart Rate Variability Summary 
Device Temperature 
Heart Rate Variability Histogram 
Heart Rate Variability Details 


In [25]:
# Importing BPM data 
# /Users/jackiekinsler/projects/sleep_analysis_py/test_dir
import os
import pandas as pd 
dfs = []
# for file in os.listdir("/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Physical_Activity"):
for file in os.listdir("/Users/jackiekinsler/projects/sleep_analysis_py/test_dir"):
    if "heart_rate-" in file: 
        print(file)
#         dfs.append(pd.read_json(f"/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Physical_Activity/{file}"))
        dfs.append(pd.read_json(f"/Users/jackiekinsler/projects/sleep_analysis_py/test_dir/{file}"))
# df = pd.concat(dfs)

df_nested = pd.concat(dfs)
df = pd.concat([df_nested.drop(['value'], axis=1), df_nested['value'].apply(pd.Series)], axis=1)
df.head()

heart_rate-2017-08-10.json
heart_rate-2018-01-13.json
heart_rate-2018-01-12.json
heart_rate-2018-01-14.json


,dateTime,bpm,confidence
0,2017-08-10 11:17:39,70,0
1,2017-08-10 11:17:45,70,0
2,2017-08-10 11:17:51,70,0
3,2017-08-10 11:17:57,70,0
4,2017-08-10 11:18:03,70,0


From the Fitbit community: 
“Confidence scores indicate confidence in the accuracy of the captured data with “3” being the highest and “0” being the lowest; a score of 0 indicates no Heart Rate is detected by the Fitbit tracker and no Heart Rate is displayed on the tracker device”.

In [37]:
# find all the values where the confidence is zero, and get the unique heartrate values 
df.head()

array([ 70.,  73.,  67.,  nan,  88.,  83.,  82.,  81.,  80.,  92.,  91.,
        94.,  75.,  74.,  65.,  57.,  69.,  68.,  97.,  96.,  90.,  89.,
        86.,  85.,  84.,  66.,  71.,  93.,  95., 124., 139., 136., 137.,
       135., 131., 127., 130., 129., 126., 125., 121., 101., 104.,  98.,
        87.,  78.,  77.,  72.,  56., 103.,  61.,  63.,  62., 105., 102.,
        99., 116., 117., 115., 108., 107., 109., 106., 100.,  79.,  48.,
        59.,  60.,  64., 110., 112., 114., 140., 144., 149., 152., 151.,
       148., 146., 143., 142., 119., 113., 122., 111.,  58., 156., 154.,
       150., 138., 141., 159., 158., 157., 155., 147., 123., 118.,  76.])

In [49]:
# Importing sleep data 
# Importing BPM data 
# /Users/jackiekinsler/projects/sleep_analysis_py/test_dir
import os
import pandas as pd 
dfs = []
# for file in os.listdir("/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Physical_Activity"):
for file in os.listdir("/Users/jackiekinsler/projects/sleep_analysis_py/test_dir"):
    if "sleep-" in file: 
        print(file)
#         dfs.append(pd.read_json(f"/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Physical_Activity/{file}"))
        dfs.append(pd.read_json(f"/Users/jackiekinsler/projects/sleep_analysis_py/test_dir/{file}"))
# df = pd.concat(dfs)

df_sleep_nested = pd.concat(dfs)

# df = pd.concat([df_nested.drop(['value'], axis=1), df_nested['value'].apply(pd.Series)], axis=1)
df_sleep_nested.head()
df_sleep_nested.loc[0,'levels']
# the summary of the sleep data seems to have everything interesting (cumulative data)
# the rest of the levels column is super detailed information about each type and duration of sleep segment 

sleep-2018-07-13.json


{'summary': {'deep': {'count': 5, 'minutes': 96, 'thirtyDayAvgMinutes': 89},
  'wake': {'count': 23, 'minutes': 58, 'thirtyDayAvgMinutes': 63},
  'light': {'count': 27, 'minutes': 228, 'thirtyDayAvgMinutes': 249},
  'rem': {'count': 5, 'minutes': 104, 'thirtyDayAvgMinutes': 82}},
 'data': [{'dateTime': '2018-08-11T23:58:00.000',
   'level': 'wake',
   'seconds': 60},
  {'dateTime': '2018-08-11T23:59:00.000', 'level': 'light', 'seconds': 210},
  {'dateTime': '2018-08-12T00:02:30.000', 'level': 'wake', 'seconds': 240},
  {'dateTime': '2018-08-12T00:06:30.000', 'level': 'light', 'seconds': 270},
  {'dateTime': '2018-08-12T00:11:00.000', 'level': 'wake', 'seconds': 390},
  {'dateTime': '2018-08-12T00:17:30.000', 'level': 'light', 'seconds': 480},
  {'dateTime': '2018-08-12T00:25:30.000', 'level': 'deep', 'seconds': 1290},
  {'dateTime': '2018-08-12T00:47:00.000', 'level': 'light', 'seconds': 870},
  {'dateTime': '2018-08-12T01:01:30.000', 'level': 'rem', 'seconds': 1170},
  {'dateTime': '2

sleep score in fitbit: 
source: https://help.fitbit.com/articles/en_US/Help_article/2439.htm



Note: shortData is only included for stages sleep logs and includes wake periods that are 3 minutes or less in duration. This distinction is to simplify graphically distinguishing short wakes from longer wakes, but they are physiologically equivalent.

from: https://dev.fitbit.com/build/reference/web-api/sleep/get-sleep-log-by-date-range/#:~:text=Note%3A%20shortData%20is%20only%20included,but%20they%20are%20physiologically%20equivalent.

sleep score is in a .csv with seemingly 1 row per day 